# Importing Packages and Libraries

In [ ]:
!pip install git+https://github.com/LIAAD/yake

  Cloning https://github.com/LIAAD/yake to /tmp/pip-req-build-ee9a_l18
  Running command git clone -q https://github.com/LIAAD/yake /tmp/pip-req-build-ee9a_l18
     |████████████████████████████████| 102kB 9.5MB/s 
  Created wheel for yake: filename=yake-0.4.4-py2.py3-none-any.whl size=59909 sha256=8711846642a1891b5abe4606011198a3c66e69344bac68ecaf0e5339158a1fb3
  Stored in directory: /tmp/pip-ephem-wheel-cache-reac5dnq/wheels/be/35/27/e4ebd54b78c1806ed8b0271ce247fcd91e2bedde35889fbc9b
  Created wheel for segtok: filename=segtok-1.5.10-cp36-none-any.whl size=25019 sha256=ce8725f5eeabedf201c241a3e9fe25415c76fc88c7ac52697fea1c2dbb785b64
  Stored in directory: /root/.cache/pip/wheels/b4/39/f6/9ca1c5cabde964d728023b5751c3a206a5c8cc40252321fb6b
Successfully built yake segtok


In [ ]:

!pip install -q sentencepiece
!pip install -q transformers
!pip install -q tokenizers
!pip install -qU hazm
!pip install -qU clean-text[gpl]




!mkdir resources
!wget -q "https://github.com/sobhe/hazm/releases/download/v0.5/resources-0.5.zip" -P resources
!unzip -qq resources/resources-0.5.zip -d resources

!pip install faiss-cpu

!rm -rf /content/4ccae468eb73bf6c4f4de3075ddb5336
!rm -rf /content/preproc
!rm preprocessing.py utils.py
!mkdir -p /content/preproc
!git clone https://gist.github.com/4ccae468eb73bf6c4f4de3075ddb5336.git /content/preproc/
!mv /content/preproc/* /content/
!rm -rf /content/preproc




     |████████████████████████████████| 1.2MB 15.8MB/s 
     |████████████████████████████████| 1.8MB 15.2MB/s 
     |████████████████████████████████| 3.2MB 52.4MB/s 
     |████████████████████████████████| 890kB 46.7MB/s 
     |████████████████████████████████| 317kB 15.8MB/s 
     |████████████████████████████████| 1.4MB 56.9MB/s 
     |████████████████████████████████| 235kB 49.6MB/s 
     |████████████████████████████████| 71kB 8.8MB/s 
     |████████████████████████████████| 133kB 15.3MB/s 
     |████████████████████████████████| 245kB 34.6MB/s 
     |████████████████████████████████| 8.2MB 13.2MB/s 
rm: cannot remove 'preprocessing.py': No such file or directory
rm: cannot remove 'utils.py': No such file or directory
Cloning into '/content/preproc'...
remote: Enumerating objects: 7, done.
remote: Total 7 (delta 0), reused 0 (delta 0), pack-reused 7
Unpacking objects: 100% (7/7), done.


In [ ]:
import numpy as np 
import pandas as pd
import re
from tqdm import tqdm 
import os
# import yake
from hazm import stopwords_list
from __future__ import unicode_literals
from hazm import *
import pickle
import requests
from termcolor import colored
from preprocessing import cleaning
import time
import hazm
from itertools import chain
# for the models
import tensorflow as tf
import matplotlib.pyplot as plt
import yake

# BERT base
from transformers import BertTokenizer, BertModel
import torch
import torch.nn as nn
import torch.nn.functional as F
from __future__ import unicode_literals
import torch.nn.functional as FloatingPointError

Base_BERT_Path = 'HooshvareLab/bert-fa-base-uncased'
import faiss


# Importing the Dataset

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

data_address = '/content/drive/MyDrive/COVID-PSS.xls'
keys_address = '/content/drive/MyDrive/keywords_final_distilled_NE (1).pickle'
cleaned_titles_address = '/content/drive/MyDrive/title_cleaned_without_corona_2.pkl'

df = pd.read_csv(data_address)
list_t = pd.read_pickle(cleaned_titles_address)

keywords = pd.read_pickle(keys_address)
keywords = [v for k,v in keywords.items()]
questions = pd.read_pickle('/content/drive/MyDrive/CoPer paper-Models/Sample Queries/Titles_with_Corona.pkl')



assert len(keywords) == len(df)
df['keywords'] = keywords
df.drop(columns=['img', 'link'], inplace=True)



Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# preparing inputs for semantic

corpora = []
for i in range(len(list_t)):

    keys = '[SEP]'.join(keywords[i])
    corpora.append(' '.join([list_t[i], keys]))

# Helper

In [ ]:
#-------------------getting some description----------#

def tokenized_info(corpora, tokenizer, config):
    """
    Gets the corpus and outputs the info related to the number of tokens
    of all records
    """

    print(f'Total number of records: {len(corpora)}')

    tokenized_corpora_lengths = [len(tokenizer.tokenize(corp)) for corp in corpora]
    max_, min_, avg_ = max(tokenized_corpora_lengths),\
                    min(tokenized_corpora_lengths),\
                    np.ceil(np.mean(tokenized_corpora_lengths))

    print(colored('The maximum length: ', 'red'), max_)
    print(colored('The minimum length: ', 'green'), min_)
    print(colored('The average length: ', 'white'), avg_)


    allowed_len = config.max_position_embeddings
    not_allowed = len([i for i in tokenized_corpora_lengths if i>allowed_len])
    print('In total ', colored(not_allowed, 'blue'), f' records\nare longer than the max_len_seq wich is {allowed_len}')


    # --------------------- The Encoder------------------#

def create_input_batches(corpora, tokenizer, 
                         batch_size=128, max_len=512):
    """
    Gets the corpora and outputs a number of batches with 
    input ids
    attention masks
    token type ids

    For the semantic search we only get the first two
    """

    all_inputs = {}

    for i in tqdm(range(0, len(corpora), batch_size),
                  position=0, leave=True):

        tokens = tokenizer.batch_encode_plus(
            corpora[i:i+batch_size],
            padding='max_length',
            truncation = True,
            max_length = max_len,
            add_special_tokens = True,
            pad_to_max_length=True,
            )
        
        all_inputs['Batch_'+ str(int(i/batch_size))] = tokens
    print('\nTotal number of batches: ', len(all_inputs))
    return all_inputs


# ------------------------ Mean pooling on GPU--------------

device = 'cuda' if torch.cuda.is_available() else 'cpu'
print('\nworking on', device)
    
def get_embeddings(all_inputs, model):
    """
    gets batches of input and outputs the mean of all tokens in a sentence
    which has 768 elements for each sentence.
    first we turn each required input batch into a tensor
    then give it to the model
    and get the first of all hidden states from it
    then we add all to mean_of_all_batches

    shape of the output:

    [# layers, # batches, # tokens, # features]
    """
    
    model_gpu = model.to(device)
    mean_of_all_batches = []

    for i in tqdm(range(len(all_inputs)), leave = True, position = 0):
        #print(f'Batch {i}')
        input_ids_batch = torch.tensor(all_inputs['Batch_'+ str(i)].input_ids)
        attention_masks_batch = torch.tensor(all_inputs['Batch_'+ str(i)].attention_mask)

        input_ids_d = input_ids_batch.to(device)
        masks_d = attention_masks_batch.to(device)

        with torch.no_grad(): 
            # print('went into no grad')
            outputs = model_gpu(input_ids_d, masks_d)  
            # print('went into the model.')
            hidden_states = outputs[2][0]
            # print('got the hidden states')
    # mean_i = torch.mean(hidden_states[0], 0)


        means_for_batch_i = []
        for j in range(len(hidden_states)):
            mean_j = torch.mean(hidden_states[j], 0)
            means_for_batch_i.append(mean_j)

        mean_of_all_batches.append(means_for_batch_i)

    mean_of_all_batches = list(chain.from_iterable(mean_of_all_batches))
    print('\nTotal number of sentences: ', len(mean_of_all_batches))
    return mean_of_all_batches
    # return mean_i


# --------------------------- query 🤔--------------------#


def get_query_embeddings(question, tokenizer,
                         model, max_len=512):

    tokenized = tokenizer.encode_plus(
        question,
        padding='max_length',
        truncation = True,
        max_length = max_len,
        add_special_tokens = True,
        pad_to_max_length=True,
        )
    tokens_tensor = torch.tensor([tokenized.input_ids])
    attention_masks = torch.tensor([tokenized.attention_mask])
    
    input_ids_d = tokens_tensor.to(device)
    masks_d = attention_masks.to(device)
    with torch.no_grad(): 
        outputs = model(input_ids_d, masks_d)  
        hidden_states = outputs[2][0]

    mean_i = torch.mean(hidden_states[0], 0)
    return mean_i



#--------------------------FAISS cosine sim for one example--------------------#

def get_FAISS_cosine_results(sent_embs, question, tokenizer,
                     model, max_len=512, top_n = 10,
                      print_results=True ):
    
    query_embeddings = get_query_embeddings(question, tokenizer,
                         model, max_len=512)
    
    index = faiss.index_factory(768, "Flat", faiss.METRIC_INNER_PRODUCT)

    emb_cor = np.array([i.cpu().numpy() for i in sent_embs])
    faiss.normalize_L2(emb_cor)
    index.add(emb_cor)

    emb_que = np.array([query_embeddings.cpu().numpy()])
    faiss.normalize_L2(emb_que)

    top_k = index.search(emb_que, top_n)

    matches = []
    if print_results == True:

        print('Top results!')
        n = 0
        for score, idx in zip(top_k[0][0], top_k[1][0]):
            print(colored(f'{n}: {idx}th', 'blue'),' corpus with score', colored( f'{score:.5f}:\n', 'blue'),  corpora[idx])
            matches.append(idx)
            n+=1

    return matches



#--------------------------FAISS L2 distance for one example--------------------#

def get_FAISS_L2_results(sent_embs, question, tokenizer,
                     model, max_len=512, top_n = 10,
                      print_results=True ):
    
    query_embeddings = get_query_embeddings(question, tokenizer,
                         model, max_len=512)
    

    emb_cor = np.array([i.cpu().numpy() for i in sent_embs])
    emb_que = np.array([query_embeddings.cpu().numpy()])


    index = faiss.IndexIDMap(faiss.IndexFlatIP(768))

    index.add_with_ids(np.array(emb_cor), np.array(range(0, len(emb_cor))))

    faiss.write_index(index, 'corona_corpora')
    index = faiss.read_index('corona_corpora')

    top_k = index.search(emb_que, top_n)

    matches = []
    if print_results == True:

        n = 0
        for score, idx in zip(top_k[0][0], top_k[1][0]):
            print(colored(f'{n}: {idx}th', 'blue'),' corpus with score', colored( f'{score:.5f}:\n', 'blue'),  corpora[idx])
            matches.append(idx)
            n+=1

    return matches


    
# phrases that need to be removed from titles
corona_phrases = ['کرونایی', 'کروناست' ,'کرونا', 'شیوع', 'بحران', 'ویروس',
                  'ویروس جدید', 'coronavirus', 'corona', 'کووید-19 ', 
                  'کووید', 'بیماری', 'بیمارانی', 'بیماران', '-۱۹', ' وی ', '19', '۱۹',
                  ' بیمار ', 'كرونا', 'كوويد', 'ويروس', r'(\s+)',]


normalizer = hazm.Normalizer()

def clean(text):
    """Cleans the titles for the semantic models"""
    for pattern in corona_phrases:
        text = re.sub(pattern, " ", text)

    text = re.sub(' +[\w] +', " ", text)
    text = normalizer.normalize(text)

    return text


#---------------------------------- get the batch results for this model-----------------------#

def get_resutls(questions, tokenizer, model, top_n):
    
    results = []
    i=0
    for question in questions:

        print(colored(f'{i}: ', 'blue'), question)
        # print('question type', type(question))
        i+=1

        # we give the cleaned question to the semantic model 
        question_cleaned = clean(question)

        question_emb = get_query_embeddings(question_cleaned, tokenizer,
                         model, max_len=512)
        
        emb_que = np.array([question_emb.cpu().numpy()])
        faiss.normalize_L2(emb_que)

        top_k = index.search(emb_que, top_n)
        indices = []
        scores = []

        # saving all the reults in a dictionary
        for score, idx in zip(top_k[0][0], top_k[1][0]):
            
            indices.append(idx)
            scores.append(score)

        results.append({'question':question,
                        'index':indices,
                        'score':scores})
    return results


working on cuda


# The Model

In [ ]:
bert_model = BertModel.from_pretrained(Base_BERT_Path,
                                 output_hidden_states = True)

bert_tokenizer = BertTokenizer.from_pretrained(Base_BERT_Path)


bert_config = AutoConfig.from_pretrained(Base_BERT_Path)
bert_model.eval()
print('Model is set on the evaluation mode.')


Model is set on the evaluation mode.


In [ ]:
tokenized_info(corpora, bert_tokenizer, bert_config)

Total number of records: 3536
The maximum length:  1746
The minimum length:  10
The average length:  124.0
In total  77  records
are longer than the max_len_seq wich is 512


In [ ]:
all_inputs_bert = create_input_batches(corpora, bert_tokenizer, batch_size=128, max_len=512)

100%|██████████| 28/28 [00:09<00:00,  3.05it/s]


Total number of batches:  28


In [ ]:
mean_of_all_sentences_bert = get_embeddings(all_inputs_bert, model=bert_model)

100%|██████████| 28/28 [02:18<00:00,  4.96s/it]


Total number of sentences:  3536


# Single Question

In [ ]:
language = "fa"
max_ngram_size = 2 # max # of words in each keyword
deduplication_thresold = 0.9 # how much they are allowed to overlap
deduplication_algo = 'seqm'
windowSize = 1 # how many words to consider when calculating the importance of 
               # each word regarding its surroundings
features = None # Whether or not to calculate a specific measurement for
                # term and word frequency
numOfKeywords = 1


def extract_keys(corpus, numOfKeywords):
    kw_extractor = yake.KeywordExtractor(lan=language, n=max_ngram_size, dedupLim=deduplication_thresold, dedupFunc=deduplication_algo, windowsSize=windowSize, top=numOfKeywords, features=features)
    keywords = kw_extractor.extract_keywords(corpus)
    # the lower the score the more relevant the key
    keys = [key[0] for key in sorted(keywords, key =  lambda x: x[1])]
    return keys

In [ ]:
question = 'واکسن تا چه حد موثر است'

extract_keys(question)

['واکسن']

In [ ]:
for question in questions:
    print(colored('query: ', 'green'), question)

    print(colored('Keyword: ', 'blue'), ' '.join(extract_keys(clean(question), 3)*3), '\n')

query:  آیا زنان باردار بیشتر در معرض خطر ابتلا به بیماری کروناهستند 
Keyword:  زنان باردار باردار بیشتر معرض خطر زنان باردار باردار بیشتر معرض خطر زنان باردار باردار بیشتر معرض خطر 

query:  من باردار هستم چگونه می‌توانم از خودم در مقابل بیماری کرونامحافظت کنم
Keyword:  محافظت کنم باردار هستم کنم محافظت کنم باردار هستم کنم محافظت کنم باردار هستم کنم 

query:  آیا زنان باردار باید برای کروناآزمایش شوند
Keyword:  زنان باردار باردار باید باید برای زنان باردار باردار باید باید برای زنان باردار باردار باید باید برای 

query:  آیا کرونامی تواند از مادر باردار به نوزاد متولد نشده جنین یا نوزاد تازه متولد شده منتقل شود
Keyword:  مادر باردار نوزاد متولد نوزاد مادر باردار نوزاد متولد نوزاد مادر باردار نوزاد متولد نوزاد 

query:  در دوران بارداری حین زایمان چه مراقبت‌هایی باید انجام شود
Keyword:  دوران بارداری بارداری حین حین زایمان دوران بارداری بارداری حین حین زایمان دوران بارداری بارداری حین حین زایمان 

query:  آیا زنان باردار مبتلا به کرونایا مشکوک به این بیماری نیاز به سزارین دارند
Keyword

In [ ]:
question = 'علائم علائم علائم '

bert_indices = get_FAISS_cosine_results(sent_embs = mean_of_all_sentences_bert,
                         question= question,
                         tokenizer=bert_tokenizer,
                         model=bert_model,
                         max_len=512,
                         top_n = 100,
                         print_results=True )

Top results!
0: 3117th  corpus with score 0.99994:
 فیلم را ببینید تصاویر جدیدی[SEP]منتشر کرده[SEP]ازمایشگاه واقع
1: 3179th  corpus with score 0.99994:
  به کانادا رسید اعلام کردند[SEP]سال چینی[SEP]حداقل جان
2: 3041th  corpus with score 0.99994:
 حریرچی منم شدم داشتم تست[SEP]امیدوارم ظرف[SEP]تست اولیها
3: 1326th  corpus with score 0.99994:
 خود درمانی در می‌تواند کشنده باشد شروع علایم[SEP]مراجعه کنند[SEP]مراکز بهداشتی
4: 1266th  corpus with score 0.99993:
 قالیباف مبتلا به شد وظایف محوله[SEP]حال حاضر[SEP]پیگیر امور
5: 1698th  corpus with score 0.99993:
 بیشترین علت فوت کودکان  فرشتگان خدمت[SEP]خدمت میکنیم[SEP]بیمارستانهای کودکان
6: 2031th  corpus with score 0.99993:
 نشانه‌های سرفه کدام‌اند؟ سرفه خشک[SEP]مجرای تنفسی[SEP]تنفسی باز
7: 2687th  corpus with score 0.99993:
 شناسایی گونه مرگبار  مورچه خوار‌های[SEP]خوار‌های پولک[SEP]انان محدود
8: 1646th  corpus with score 0.99993:
 هشدار درباره واکسن عجله نکنید احتمالی پخش[SEP]انتظارهای سیاسی[SEP]تجاری بسیاری
9: 1219th  corpus with score 0.999

# Results in Batches

In [ ]:


index = faiss.index_factory(768, "Flat", faiss.METRIC_INNER_PRODUCT)
emb_cor = np.array([i.cpu().numpy() for i in mean_of_all_sentences_bert])
faiss.normalize_L2(emb_cor)
index.add(emb_cor)


results = get_resutls(questions, tokenizer=bert_tokenizer, model=bert_model, top_n=10)

0:  آیا زنان باردار بیشتر در معرض خطر ابتلا به بیماری کروناهستند 
1:  من باردار هستم چگونه می‌توانم از خودم در مقابل بیماری کرونامحافظت کنم
2:  آیا زنان باردار باید برای کروناآزمایش شوند
3:  آیا کرونامی تواند از مادر باردار به نوزاد متولد نشده جنین یا نوزاد تازه متولد شده منتقل شود
4:  در دوران بارداری حین زایمان چه مراقبت‌هایی باید انجام شود
5:  آیا زنان باردار مبتلا به کرونایا مشکوک به این بیماری نیاز به سزارین دارند
6:  آیا مادر مبتلا به کرونامی تواند به نوزاد خود شیر بدهد
7:  آیا در صورت مبتلا شدن به کووید۱۹ می‌توانم کودک خود را در آغوش بگیرم
8:  من به کرونامبتلا هستم حال عمومی مساعدی ندارم که بتوانم به کودک خود مستقیما شیر بدهم چه باید بکنم
9:  آیا خشک­ کن­های دست در نابودی ویروس جدید کرونا موثر هستند
10:  آیا گندزداییضدعفونی با لامپ ماورای بنفش ویروس جدید کرونا را از بین می­برد
11:  آیا سنجش حرارت بدن اندازه گیری تب در شنسایی افراد مبتلا به عفونت ویروس جدید کرونا موثر است
12:  آیا اسپری کردن الکل یا محلول‌های حاوی کلر روی تمام بدن می­تواند ویروس جدید کرونا را نابود کند
13:  آیا و

In [ ]:
for i in range(len(questions)):
    print(results[i]['question'])

    for j in range(len(results[i]['index'])):
        print(colored(f'{j}: ', 'blue'), corpora[results[i]['index'][j]])
        print(colored('score', 'blue'), results[i]['score'][j])
        

آیا زنان باردار بیشتر در معرض خطر ابتلا به بیماری کروناهستند 
0:  سیگاری‌ها بیشتر در خطر ابتلا به هستند معرض ابتلا[SEP]درگیر میکند[SEP]مصرف دخانیات
score 0.99996006
1:   به کانادا رسید اعلام کردند[SEP]سال چینی[SEP]حداقل جان
score 0.99994695
2:  مینو محرز به مبتلا شد دلیل ابتلا[SEP]اساس پیگیریها[SEP]عمومی محرز
score 0.99994606
3:  چگونه خطر ابتلا به را در آرایشگاه‌ها کاهش دهیم؟ روزهای ابتدایی[SEP]گذشته نمیتوانستیم[SEP]زودی ریشهکن
score 0.9999456
4:  شیردهی بارداری مادران مبتلا به  علایم مشکوک[SEP]مراقبتهای لازم[SEP]منتقل نشود
score 0.9999427
5:  آیا کودکان دچار عوارض شدید می‌شوند؟ ازمایش برای[SEP]nhk world[SEP]بین زیر
score 0.99994236
6:  قالیباف مبتلا به شد وظایف محوله[SEP]حال حاضر[SEP]پیگیر امور
score 0.99994206
7:  راه‌های مقابله با در سفر پمپ بنزین[SEP]مصرف کنید[SEP]کرده باشید
score 0.9999416
8:  تاثیر مستقیم خواب بر ابتلا به  همچنین توصیه[SEP]زنگ گوشی[SEP]توصیه کرد
score 0.9999415
9:  نحوه نگهداری مبتلا به در منزل هفته طول[SEP]علایم خفیف[SEP]تشدید علایمی
score 0.9999409
من باردار ه

# Save the Results

In [ ]:
with open('/content/drive/MyDrive/CoPer paper-Models/Results/BERT.pkl', 'wb') as f:
    pickle.dump(results, f)